    000_16_230114_141_subway_transer_21
    
    data : 021_경춘선_주소데이터_20221122.csv
     save : subway_info_21.csv
     display : subway_info_21.html

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/021_경춘선_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,코레일,경춘,가평(자라섬·남이섬),경기도 가평군 가평읍 달전리 567,경기도 가평군 가평읍 문화로 13-42
1,코레일,경춘,갈매,구리시 갈매동 502-39,구리시 경춘북로 229
2,코레일,경춘,강촌,강원도 춘천시 남산면 방곡리 409,강원도 춘천시 남산면 강촌로 150
3,코레일,경춘,광운대,서울특별시 노원구 월계동 85,서울특별시 노원구 석계로 98-2
4,코레일,경춘,굴봉산(제이드가든),강원도 춘천시 남산면 백양리 588-30,강원도 춘천시 남산면 서백길 192


In [5]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  25 non-null     object
 1   선명       25 non-null     object
 2   역명       25 non-null     object
 3   지번주소     25 non-null     object
 4   도로명주소    25 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [6]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [7]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [8]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["지번주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["지번주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,코레일,경춘,가평(자라섬·남이섬),경기도 가평군 가평읍 달전리 567,경기도 가평군 가평읍 문화로 13-42,1000909.30834495,1979371.37898815
1,코레일,경춘,갈매,구리시 갈매동 502-39,구리시 경춘북로 229,965992.007098339,1959467.41640435
2,코레일,경춘,강촌,강원도 춘천시 남산면 방곡리 409,강원도 춘천시 남산면 강촌로 150,1011812.67104266,1978483.14492289
3,코레일,경춘,광운대,서울특별시 노원구 월계동 85,서울특별시 노원구 석계로 98-2,961421.292738243,1958207.60975323
4,코레일,경춘,굴봉산(제이드가든),강원도 춘천시 남산면 백양리 588-30,강원도 춘천시 남산면 서백길 192,1005077.72402026,1981374.22003571


In [9]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [10]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,경춘,가평(자라섬·남이섬),경기도 가평군 가평읍 달전리 567,경기도 가평군 가평읍 문화로 13-42,1000909.30834495,1979371.37898815,127.510331,37.814073
1,코레일,경춘,갈매,구리시 갈매동 502-39,구리시 경춘북로 229,965992.007098339,1959467.41640435,127.114562,37.634043
2,코레일,경춘,강촌,강원도 춘천시 남산면 방곡리 409,강원도 춘천시 남산면 강촌로 150,1011812.67104266,1978483.14492289,127.634192,37.805991
3,코레일,경춘,광운대,서울특별시 노원구 월계동 85,서울특별시 노원구 석계로 98-2,961421.292738243,1958207.60975323,127.062826,37.622507
4,코레일,경춘,굴봉산(제이드가든),강원도 춘천시 남산면 백양리 588-30,강원도 춘천시 남산면 서백길 192,1005077.72402026,1981374.22003571,127.557703,37.832111


In [11]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [12]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [13]:
#5 seoul university 
map

In [14]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_21.html")


In [15]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_21.csv", index = False)
